# Using the Materials API

### Presented by: John Dagdelen

In this module, we will cover:

* Getting your Materials Project API key.
* The `pymatgen.MPRester` built-ins.
* The `MPRester.query` method for accessing the MP database.
* The [mapidoc](https://github.com/materialsproject/mapidoc) (Materials Project documentation) repository.
* Accessing the API more directly, with plain HTTP requests, via the Python `requests` library.
* Examples of API usage to get chemical system entries for generating phase diagrams and calculating reaction energies.

## Getting an API key

Go to https://materialsproject.org/dashboard. If you see 'None' under the 'Generate API key' button, click the button. Highlight and copy the string under the button -- that is your API key. Paste it in as the value inside the quotation marks ("") below.

In [ ]:
import os
os.environ['MAPI_KEY'] = "YOUR_API_KEY"

## MPRester
* Open pymatgen.MPRester web documentation
* get feet wet with calling a few of the "specialty" methods
* introduce the powerful `query` method

## `mapidoc` repo
* Go over first part of README
* examples of MongoDB syntax
* search for properties, more MongoDB syntax
* Go over remainder of README -- examples of not using pymatgen for API queries

## generating phase diagrams - plotting, calculating phase equilibria properties

examples from hackingmaterials.com and pymatgen.org

In [ ]:
%matplotlib inline

from pymatgen import MPRester
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
from pymatgen.phasediagram.analyzer import PDAnalyzer
from pymatgen.phasediagram.maker import PhaseDiagram
from pymatgen.phasediagram.plotter import PDPlotter

In [ ]:
system = ["Fe", "P"]

In [ ]:
mpr = MPRester() # object for connecting to MP REST interface

Set energy corrections and +U/pseudopotential choice:

In [ ]:
compat = MaterialsProjectCompatibility()

Create phase diagram by fetching unprocessed entries from MP,

In [ ]:
unprocessed_entries = mpr.get_entries_in_chemsys(system)

filtering and adding energy corrections,

In [ ]:
processed_entries = compat.process_entries(unprocessed_entries)

and, finally, instantiating a Phase Diagram object from thge processed entries:

In [ ]:
pd = PhaseDiagram(processed_entries)

Let's plot the phase diagram.

In [ ]:
plotter = PDPlotter(pd, show_unstable=0)
plotter.show()

In [ ]:
plotter = PDPlotter(pd, show_unstable=0.100)
plotter.show()

Let's analyze the phase diagram to list stable and unstable entries.

In [ ]:
pda = PDAnalyzer(pd)

A table of stable entries:

In [ ]:
for e in pd.stable_entries:
    print("{}\t{}".format(e.composition.reduced_formula, e.entry_id))

Now, let's print a table of unstable entries, inlcuding their energy above hull and predicted decomposition. Since this is more involved display-wise, we'll use a handy and popular library for dealing with data frames and displaying them nicely inline.

In [ ]:
from pandas import DataFrame

In [ ]:
import collections

data = collections.defaultdict(list)
for e in pd.unstable_entries:
    decomp, ehull = pda.get_decomp_and_e_above_hull(e)
    data["Materials ID"].append(e.entry_id)
    data["Composition"].append(e.composition.reduced_formula)
    data["Ehull"].append(ehull)    
    data["Decomposition"].append(
        " + ".join(["{:.2f} {} [{}]".format(
                    v, k.composition.reduced_formula, k.entry_id)
                    for k, v in decomp.items()]))

from pandas import DataFrame
df = DataFrame(data, columns=[
        "Materials ID", "Composition", "Ehull", "Decomposition"])

df

In [ ]:
mpr.query('mp-27164', ['e_above_hull'])

Let's examine another chemical system: C-Ca-O.

In [ ]:
entries = a.get_entries_in_chemsys(['Ca', 'C', 'O'])
pd = PhaseDiagram(compat.process_entries(entries))

In [ ]:
plotter = PDPlotter(pd, show_unstable=True)
plotter.show()

In [ ]:
pda = PDAnalyzer(pd)

And let's print out all entries now, stable and unstable.

In [ ]:
import collections

data = collections.defaultdict(list)
for e in pd.all_entries:
    decomp, ehull = pda.get_decomp_and_e_above_hull(e)
    data["Materials ID"].append(e.entry_id)
    data["Composition"].append(e.composition.reduced_formula)
    data["Ehull"].append(ehull)    
    data["Decomposition"].append(
        " + ".join(["{:.2f} {} [{}]".format(
                    v, k.composition.reduced_formula, k.entry_id)
                    for k, v in decomp.items()]))

from pandas import DataFrame
df = DataFrame(data, columns=[
        "Materials ID", "Composition", "Ehull", "Decomposition"])

df

## Calculating reaction energies
pymatgen.org example

In [ ]:
from pymatgen.matproj.rest import MPRester
from pymatgen import Composition
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.core.units import FloatWithUnit
from pymatgen.analysis.reaction_calculator import ComputedReaction

#This initializes the REST adaptor. Put your own API key in.
a = MPRester()

#This gets all entries belonging to the Ca-C-O system.
all_entries = a.get_entries_in_chemsys(['Ca', 'C', 'O'])

#This method simply gets the lowest energy entry
#for all entries with the same composition.
def get_most_stable_entry(formula):
    relevant_entries = [
        entry for entry in all_entries
        if entry.composition.reduced_formula == Composition(formula).reduced_formula]
    relevant_entries = sorted(relevant_entries, key=lambda e: e.energy_per_atom)
    return relevant_entries[0]

CaO = get_most_stable_entry("CaO")
CO2 = get_most_stable_entry("CO2")
CaCO3 = get_most_stable_entry("CaCO3")

reaction = ComputedReaction([CaO, CO2], [CaCO3])
energy = FloatWithUnit(reaction.calculated_reaction_energy, "eV atom^-1")

print "Caculated"
print reaction
print "Reaction energy = {}".format(energy.to("kJ mol^-1"))
print

# The following portions demonstrate how to get the experimental values as well.
exp_CaO = a.get_exp_entry("CaO")
exp_CaCO3 = a.get_exp_entry("CaCO3")

#Unfortunately, the Materials Project database does not have
# gas phase experimental entries.
# This is the value from NIST. We manually create the entry.
#Exp entries should be in kJ/mol.
exp_CO2 = ComputedEntry("CO2", -393.51)

exp_reaction = ComputedReaction([exp_CaO, exp_CO2], [exp_CaCO3])

print "Experimental"
print exp_reaction
print "Reaction energy = {} kJ mol^-1".format(
    exp_reaction.calculated_reaction_energy)